# Хакатон "Сибур". Команда Hack.zamAI

- метрики изношенности (пробег каждого вагона)

- почему нельзя все типы ремонта за одну ближайщую дату?
- время на ремонт?
- 

### Задача "Вагоны"  

Разработать комплексное решение, помогающее специалисту по расчету календаря отправки вагонов на плановое тех.обслуживание оптимизировать логистические затраты.

Решение должно быть полезно для текущей работы сотрудника, упрощать его работу, помогать принимать оперативные решения.

Решение должно состоять из двух частей:
1. Алгоритм на основе анализа данных, предсказывающий оптимальные времена и депо для ремонтов.
2. Интерфейс с визуализацией данных и результатов работы алгоритма и любых дополнительных данных. Это может быть web, mob app, bot и другие формы.

Главный комментарий: это реальный кейс, решение должно быть полезно реальному человеку с реальными должностными обязанностями в реальных рабочих условиях.

Для решения задачи командам будет предложено:
1. Датасет об отправке на ремонт 18000 вагонов за несколько лет с данными о тайминге, расстояниях и т.д.
https://drive.google.com/open?id=1FGFVbgqe5QWilpiyVG9z3AhVRN7Bt_Mp

Уточнения к датасету: 
1.1 перед каждым ремонтом нужно потратить 3 000 000 р  на подготовку вагона. При этом считается, что если делается 2 ремонта подряд, то достаточно сделать 1 подготовку. 

1.2 если вагон шел из станции А к станции Б, и мы решили сделать ремонт, то вагон должен попасть из пункта А в депо и обратно в пункт А, только после этого можно продолжить маршрут. Работает допущение, что путь в депо происходит мгновенно, но стоит 1000 р за км. 
В качестве усложнения задания вы можете вычислить, сколько занимает маршрут между каждой парой станций и убрать допущение о том, что транспортировка в депо происходит мгновенно. В таком случае, мы не на каждом маршруте можем попасть в депо, а только в тех, где путь из А -> депо -> А -> Б укладывается в расписание. 

1.3 Различные ремонты независимы. Это значит, например, то, что капитальный ремонт не отменяет плановый предупредительный ремонт. 

1.4 Каждый ремонт необходимо повторять раз в 2 года. То есть, если в датасете указано, что ремонт запланирован в 2017-01-01, то его нужно повторить не позднее, чем 2019-01-01.
Мы считаем, что можем оптимизировать только то, что идет после 1-го мая 2018 года. До 1-го мая все ремонты – свершившийся факт, на который мы не можем повлиять

2. Код для вычисления стоимости ремонта для 1 вагона: https://drive.google.com/file/d/1XhgnNK_oEOj1aIMbYKC7fy6VSUE762dm/view?usp=sharing

3. Пример того, как решение выглядит сейчас: https://drive.google.com/file/d/1UH00W1RTztun-vd8agxqcxFSNATPISJh/view?usp=sharing

https://drive.google.com/file/d/1BvalSlBBIRoRLLV7zgOAbJ-b0YneylvY/view?usp=sharing


4. Живое общение с бизнес-оунером процесса.


### Data exploration

In [303]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from tqdm import tqdm
from datetime import datetime
from dateutil.relativedelta import relativedelta
import json
import random
from decimal import Decimal

%matplotlib inline

In [186]:
depos = pd.read_csv('./data/Депо.csv')
stations = pd.read_csv('./data/Станции.csv')

distances = pd.read_csv('./data/dists.csv')
movements = pd.read_csv('./data/Передвижения вагонов по датам.csv')
repairs = pd.read_csv('./data/Вагоны и плановые ремонты.csv')
repair_prices = pd.read_csv('./data/Плановые ремонты со стоимостями.csv')

До конца не понятно, что делать с пропусками, ведь мы знаем только по одной дате для каждого ремонта для каждого вагона, то есть не можем "отсчитать" дату для определённого ремонта для данного вагона, если раньше его в данных не было.  
Однако в дальнейшей реализации очень пригодится иметь пропуски как очень далёкие даты, заполним вот так:

In [213]:
movements.fillna(value='5000-01-01', inplace=True)
repairs.fillna(value='5000-01-01', inplace=True);

Первые два датасета по сути не нужны, они есть в таблице distances.

In [214]:
depos

,id,depo_name
0,1,Депо-Ленинское
1,2,Депо-Комсомольское
2,3,Депо-Молодежное


In [215]:
stations

,id,name
0,1,Сосновка
1,2,Липовка
2,3,Рожок
3,4,Гатка
4,5,Георгополь
5,6,Приморск
6,7,Милта
7,8,Новорепное
8,9,Ясная поляна
9,10,Северный


Дальше идут полезные данные:

In [216]:
repair_prices

,id,repair_name,repair_cost
0,1,Капитальный ремонт,94000000
1,2,Деповской ремонт,19000000
2,3,Плановый предупредительный ремонт,13000000
3,4,Вакуумная очистка и гидроиспытания (ВОиГИ),18000000


In [217]:
distances

,station_id,name,depo_id,depo_name,distance_to_depo
0,1,Сосновка,1,Депо-Ленинское,10077
1,1,Сосновка,2,Депо-Комсомольское,10436
2,1,Сосновка,3,Депо-Молодежное,10522
3,2,Липовка,1,Депо-Ленинское,2084
4,2,Липовка,2,Депо-Комсомольское,3789
5,2,Липовка,3,Депо-Молодежное,7637
6,3,Рожок,1,Депо-Ленинское,9936
7,3,Рожок,2,Депо-Комсомольское,2341
8,3,Рожок,3,Депо-Молодежное,4198
9,4,Гатка,1,Депо-Ленинское,7157


Сразу предобработаем так, чтобы из каждой станции путь был в оптимальное депо:

In [218]:
opt_dist = distances.groupby([pd.Grouper(key='station_id')])['distance_to_depo'].min()
opt_dist = pd.DataFrame(opt_dist)
opt_dist['station_id'] = opt_dist.index
opt_dist.index = np.arange(len(opt_dist))
#df['depo_name'] = distances[df['distance_to_depo'] == distances['distance_to_depo']]['depo_name']
opt_dist

,distance_to_depo,station_id
0,10077,1
1,2084,2
2,2341,3
3,3029,4
4,3662,5
5,2238,6
6,5698,7
7,4816,8
8,1779,9
9,1431,10


In [219]:
movements

,car_num,date,station_id
0,57463085,2015-01-01,2
1,57463085,2015-01-25,6
2,57463085,2015-02-18,9
3,57463085,2015-03-14,5
4,57463085,2015-04-07,8
5,57463085,2015-05-01,2
6,57463085,2015-05-25,6
7,57463085,2015-06-18,9
8,57463085,2015-07-12,3
9,57463085,2015-08-05,6


In [220]:
repairs

,car_num,ct_name,psevdoname,std_kap,std_ppr,std_dep,std_vogi
0,57463085,Газовые,15-78-5,2021-08-08,2018-08-26,2019-08-31,2019-05-19
1,57456220,Газовые,15-78-5,2021-08-02,2019-05-19,2019-09-15,2019-05-19
2,57453656,Газовые,15-78-5,2021-07-19,2019-01-22,2019-07-07,2019-05-18
3,57463507,Газовые,15-78-5,2021-07-19,2019-03-18,2018-06-02,2019-05-18
4,57463309,Газовые,15-78-5,2021-06-20,2018-05-16,2019-01-09,2019-04-24
5,57463341,Газовые,15-78-5,2021-07-11,2018-08-02,2018-06-14,2019-05-20
6,57463275,Газовые,15-78-5,2021-06-20,2018-09-11,2018-10-03,2019-04-22
7,57463317,Газовые,15-78-5,2021-06-14,2018-12-15,2019-06-15,2019-04-22
8,57457038,Газовые,15-78-5,2018-11-25,2018-10-24,2021-11-03,2024-10-24
9,57463523,Газовые,15-78-5,2021-07-01,2019-04-13,2019-02-28,2019-04-22


Код из выданного ноутбука (стоимость одной ремонтной итерации для одного вагона):

In [221]:
repair_prices = {'std_kap': 94000000, 
                 'std_ppr': 19000000, 
                 'std_dep': 13000000, 
                 'std_vogi': 18000000, 
                 'preparation': 3000000}
price_for_km = 1000

In [222]:
def get_station_from_date(date):
    for index, (car_num, station_date, station_id) in movements.iterrows():
        if station_date < date:
            return station_id
    assert False, 'No stations before this date'

total_repair_price = 0
for repair_type in ['std_kap', 'std_ppr', 'std_dep', 'std_vogi']:
    repair_date = repairs[repair_type].values[0]
    station_id = get_station_from_date(repair_date)
    min_distance_to_depo = distances[distances['station_id'] == station_id]['distance_to_depo'].min()
    total_repair_price += 2 * min_distance_to_depo * price_for_km + \
            repair_prices[repair_type] + \
            repair_prices['preparation']

total_repair_price

172672000

Сначала посчитаем, сколько сейчас стоит ремонт для каждого вагона и суммарно (без оптимизаций, следуя дедайнам в исходной таблице):

In [223]:
start_date = '2018-05-01'  # начало оптимизации - 1 мая 2018
end_date = '2018-05-31'  # конец оптимизации - 31 мая 2018

Маска для выделения данных, попадающих во временные рамки:

In [224]:
REPAIR_TYPES = ['std_kap', 'std_ppr', 'std_dep', 'std_vogi']

In [225]:
kap = np.logical_and(repairs['std_kap'] >= start_date, repairs['std_kap'] <= end_date)
dep = np.logical_and(repairs['std_dep'] >= start_date, repairs['std_dep'] <= end_date)
ppr = np.logical_and(repairs['std_ppr'] >= start_date, repairs['std_ppr'] <= end_date)
vogi = np.logical_and(repairs['std_vogi'] >= start_date, repairs['std_vogi'] <= end_date)
mask = np.logical_or(np.logical_or(np.logical_or(kap, dep), ppr), vogi)

In [226]:
actual_repairs = repairs[mask]

## ХРАНИТЬ, В КАКОЕ ДЕПО ЕДЕТ ПРИ ВЫБОРЕ ПУТИ!

In [227]:
def calculate_price_vagon(df, car_num, st_date, e_date):
    car_df = df[df['car_num'] == car_num]
    
    def get_station_from_date(car_num, date):
        car_movements = movements[movements['car_num'] == car_num]
        for index, (car_num, station_date, station_id) in car_movements.iterrows():
            if station_date < date:
                return station_id

    total_repair_price = 0
    for repair_type in REPAIR_TYPES:
        repair_date = car_df[repair_type].values[0]
        if st_date <= repair_date and repair_date <= e_date:
            station_id = get_station_from_date(car_num, repair_date)
            min_distance_to_depo = opt_dist[opt_dist['station_id'] == station_id]['distance_to_depo'].values[0]
            total_repair_price += 2 * min_distance_to_depo * price_for_km + \
                    repair_prices[repair_type] + \
                    repair_prices['preparation']

    return total_repair_price

In [228]:
calculate_price_vagon(actual_repairs, 57463309, start_date, end_date)

26168000

Это число = стоимость ремонтов вагона номер 57463309 в мае 2018, если мы отправили его в самое выгодное (для станции) депо из станции 2 в день 2015-01-01

Посчитаем суммарную стоимость ремонта всех вагонов (которые имеют ремонт по расписанию в мае) за май 2018 года:

In [148]:
def calculate_price_all(df, st_date, e_date):
    total_repair_price = 0
    for car_num in tqdm(np.unique(df['car_num'])):
        total_repair_price += calculate_price_vagon(df, car_num, st_date, e_date)
    return total_repair_price

In [86]:
%%time
calculate_price_all(actual_repairs, start_date, end_date)

100%|██████████████████████████████████████| 1063/1063 [06:42<00:00,  2.64it/s]


Wall time: 6min 42s


38897616000

In [88]:
total_cost_may18 = 38897616000

### Идеи

* Предобработать алгоритмом Флойда пути от станций до других станций (см. другой пункт)
* Написать формулу: оптимизация общей суммы = оптимизация слагаемых
* Есть 24 месяца, то есть ~24 "хороших" варианта ремонтов
* Капитальный ремонт 1 раз в 2 года
* Состояние: (дата, место, какие ремонты делать), метрика (эвристика) хода - расчёт денег при движении вагона из опр. станции в депо в эту дату, учитывающий дедлайновую дату ремонта:  
\__ЭВРИСТИКИ\__ (*А-Star, Метод ветвей и границ, Альфа-бета отсечение*):  
- слишком частые кап ремонты
- много времени до дедлайна ремонта
- уже не идём дальше по ветви, если в вершине большая сумма (больше бейзлайна)
- не укладывается в расписание
- бейзлайн - ремонты в день дедлайна
- невозможность попасть из А в Б на основе просчёта алгоритмом Флойда-Уоршалла
- 

- TASK: сколько мы сэкономили для каждого вагона и в целом
- по неделям
- по вагонам
- общую

### Первая оптимизация - совмещение нескольких типов ремонта в один день

*Идея*: Есть несколько типов ремонта, все обычно проводятся в разные даты. Перегнать вагон в депо стоит денег, подготовить вагон к ремонту тоже стоит денег. Возникает разумная мысль - почему бы не сделать некоторые типы ремонтов в один день, так мы будем экономить на подготовке вагона к ремонту и на логистике - не нужно ездить в депо несколько раз.

In [285]:
def calculate_price_vagon_opt1(df, car_num, st_date, e_date, timedelta=30):
    '''
        timedelta - этим параметром регулируем то, 
        как сильно ремонты могут отстоять друг от друга по датам 
        (максимальное количество дней в разнице дат ремонтов)
    '''
    car_df = df[df['car_num'] == car_num]

    def get_station_from_date(car_num, date):
        car_movements = movements[movements['car_num'] == car_num]
        for index, (car_num, station_date, station_id) in car_movements.iterrows():
            if station_date < date:
                return station_id

    reps = {}
    for repair_type in REPAIR_TYPES:
        reps[repair_type] = car_df[repair_type].values[0]
    # сортируем типы ремонтов для данного вагона по датам -
    # к самому раннему будем сливать (надо подумать, может выгоднее по-другому)
    sorted_by_dates = sorted(reps.items(), key=lambda x:x[1])

    types = [x[0] for x in sorted_by_dates]
    dates = [x[1] for x in sorted_by_dates]
    together = set([types[0]])
    # ПРОРАБОТАТЬ БОЛЬШЕ СЛУЧАЕВ!
    # в данном случае идёт слияние только !_самого_раннего_по_дате_типа_ремонта_!, и какого-то ещё;
    # по-хорошему надо учесть случаи, когда любой тип с любым может быть слит
    for i in range(1, len(dates)):
        first_date = datetime.strptime(dates[0], '%Y-%m-%d')
        second_date = datetime.strptime(dates[i], '%Y-%m-%d')
        diff = relativedelta(second_date, first_date)
        if diff.years == 0 and diff.months == 0 and diff.days < timedelta:
            together.add(types[i])  # если разница между датами ремонтов меньше timedelta, то ремонтируем в один день
    total_repair_price = 0
    gain = 0  # выигрыш по сравнени с решением без слияния
    for repair_type in REPAIR_TYPES:
        repair_date = car_df[repair_type].values[0]
        if st_date <= repair_date and repair_date <= e_date:
            station_id = get_station_from_date(car_num, repair_date)
            min_distance_to_depo = opt_dist[opt_dist['station_id'] == station_id]['distance_to_depo'].values[0]
            # hardcode: если этот тип ремонта слит с самым ранним, то мы экономим gain
            if repair_type != types[0] and repair_type in together:
                gain += repair_prices['preparation'] * (len(together) - 1) + \
                        2 * min_distance_to_depo * price_for_km
                continue
            total_repair_price += 2 * min_distance_to_depo * price_for_km + \
                    repair_prices[repair_type] + \
                    repair_prices['preparation']

    return total_repair_price, gain

In [286]:
calculate_price_vagon_opt1(actual_repairs, 57463309, start_date, end_date)

(26168000, 0)

In [287]:
def calculate_price_all_opt1(df, st_date, e_date):
    total_repair_price = 0
    total_gain = 0
    for car_num in tqdm(np.unique(df['car_num'])):
        tmp_price, tmp_gain = calculate_price_vagon_opt1(df, car_num, st_date, e_date)
        total_repair_price += tmp_price
        total_gain += tmp_gain
    return total_repair_price, total_gain

In [288]:
%%time
calculate_price_all_opt1(actual_repairs, start_date, end_date)

100%|██████████████████████████████████████| 1063/1063 [00:11<00:00, 95.94it/s]


Wall time: 11.1 s


(34283744000, 1112872000)

### Вторая оптимизация - перегон вагона в другую станцию

*Идея*: В предоставленном коде рассчитывается расстояние от станции до депо, однако станция берётся самая ранняя (`if station_date < date: return station_id`). Но может быть куда выгоднее отправить вагон на ремонт из другой станции, из которой до депо ближе (естественно, если вагон находится в этой станции до даты ремонта). Также будем заботиться о том, чтобы перегон вагона из станции до депо и обратно не мешал вагону отправится из этой станции в следующую.

In [292]:
def calculate_price_vagon_opt2(df, car_num, st_date, e_date, timedelta=30):
    '''
        Принципиальное отличие от opt1 - функция get_station_from_date().
        
        timedelta - этим параметром регулируем то, 
        как сильно ремонты могут отстоять друг от друга по датам 
        (максимальное количество дней в разнице дат ремонтов)
    '''
    car_df = df[df['car_num'] == car_num]

    def get_station_from_date(car_num, date):
        '''
            Учёт того, с какой станции лучше поехать в депо
            TODO: с каких-то нельзя, можно не успеть по расписанию, учесть и это
        '''
        opt_station_id = -1
        min_dist = 1e9
        car_movements = movements[movements['car_num'] == car_num]
        for index, (car_num, station_date, station_id) in car_movements.iterrows():
            # если дата нахождения на этой станции < даты ремонта
            if station_date < date:
                dist = opt_dist[opt_dist['station_id'] == station_id]['distance_to_depo'].values[0]
                # берём минимальную по расстоянию до депо среди всех доступных станций
                if dist < min_dist:
                    min_dist = dist
                    opt_station_id = station_id
        return opt_station_id

    reps = {}
    for repair_type in REPAIR_TYPES:
        reps[repair_type] = car_df[repair_type].values[0]
    # сортируем типы ремонтов для данного вагона по датам -
    # к самому раннему будем сливать (надо подумать, может выгоднее по-другому)
    sorted_by_dates = sorted(reps.items(), key=lambda x:x[1])

    types = [x[0] for x in sorted_by_dates]
    dates = [x[1] for x in sorted_by_dates]
    together = set([types[0]])
    # ПРОРАБОТАТЬ БОЛЬШЕ СЛУЧАЕВ!
    # в данном случае идёт слияние только !_самого_раннего_по_дате_типа_ремонта_!, и какого-то ещё;
    # по-хорошему надо учесть случаи, когда любой тип с любым может быть слит
    for i in range(1, len(dates)):
        first_date = datetime.strptime(dates[0], '%Y-%m-%d')
        second_date = datetime.strptime(dates[i], '%Y-%m-%d')
        diff = relativedelta(second_date, first_date)
        if diff.years == 0 and diff.months == 0 and diff.days < timedelta:
            together.add(types[i])  # если разница между датами ремонтов меньше timedelta, то ремонтируем в один день
    total_repair_price = 0
    gain = 0  # выигрыш по сравнени с решением без слияния
    for repair_type in REPAIR_TYPES:
        repair_date = car_df[repair_type].values[0]
        if st_date <= repair_date and repair_date <= e_date:
            station_id = get_station_from_date(car_num, repair_date)
            min_distance_to_depo = opt_dist[opt_dist['station_id'] == station_id]['distance_to_depo'].values[0]
            # hardcode: если этот тип ремонта слит с самым ранним, то мы экономим gain
            if repair_type != types[0] and repair_type in together:
                gain += repair_prices['preparation'] * (len(together) - 1) + \
                        2 * min_distance_to_depo * price_for_km
                continue
            total_repair_price += 2 * min_distance_to_depo * price_for_km + \
                    repair_prices[repair_type] + \
                    repair_prices['preparation']

    return total_repair_price, gain

In [293]:
calculate_price_vagon_opt2(actual_repairs, 57463309, start_date, end_date)

(24862000, 0)

In [294]:
def calculate_price_all_opt2(df, st_date, e_date):
    total_repair_price = 0
    total_gain = 0
    for car_num in tqdm(np.unique(df['car_num'])):
        tmp_price, tmp_gain = calculate_price_vagon_opt1(df, car_num, st_date, e_date)
        total_repair_price += tmp_price
        total_gain += tmp_gain
    return total_repair_price, total_gain

In [295]:
%%time
calculate_price_all_opt2(actual_repairs, start_date, end_date)

100%|██████████████████████████████████████| 1063/1063 [00:11<00:00, 91.55it/s]


Wall time: 11.6 s


(34283744000, 1112872000)

### Продвинутая оптимизация - альфа-бета отсечение, алгортим А*

*Идея:*  

**Подход А*  **  
Можно описать всё происходящее с переждвием вагонов как некую игру. В ней нужно объявить состояния, например, состояние = кортеж (дата, место (станция), ремонт1 (bool), ремонт2 (bool), ремонт3 (bool), ремонт4 (bool)). Переход в каждое новое состояние характеризуется числом - количество денег, которые мы сэкономили/потратили. Это число считается с помощью специальных эвристических функций, учитывающих:  
- слишком частые кап ремонты
- много времени до дедлайна ремонта
- уже не идём дальше по ветви, если в вершине большая сумма (больше бейзлайна)
- не укладывается в расписание
- бейзлайн - ремонты в день дедлайна
- невозможность попасть из А в Б на основе просчёта алгоритмом Флойда-Уоршалла  


**Подход методом ветвей и границ (альфа-бета отсечение)**  
Можно представить выбор ремонтирвоать/не ремонтировать каждый месяц определённый вагон с помощью дерева, у котрого каждое состояние делится на ещё несколько. Далее использутся примерно те же эвристики, позволяя "отсекать" заведомо невыгодные состояния и находя оптимальные

### Экспорт данных в веб-интерфейс

Пример json, который нужно вернуть (неоптимизированный маршрут):

In [ ]:
{'breaks': 3,  # поломки, = randint(0, 5) 
 'distance_left': 65,  # время до ближайшего ремонта / 2 года
 'distance_ran': 98765,  # 1 - distance_left
 'id': '1488228228',  # car_num
 'location': 'Галич',  # station
 'type': 'Химические',
 'maintainces':  # ближайшие ремонты (?)
 [{'date': '22.03.2018',
   'depo': 'Депо 1',
   'maintaince_type': 'ОУМК'},
  {'date': '27.03.2018', 
   'depo': 'Депо 2', 
   'maintaince_type': 'ОУМК2'}],
 'part_of_time_to_next_mainaince_required': 0.9,  # время до ближайшего ремонта / 2 года (0..100)
 'schedule': 
 [{'date': '22.02.2018', 'from': 'Галич', 'to': 'Иваново'},
  {'date': '23.02.2018', 'from': 'Иваново', 'to': 'Помпеи'}], # поездки вагона, выдать ~7 элементов
 'total_weight_transported': 143}  # Сгенерить файл для каждого вагона 

In [357]:
import io

In [395]:
def make_json_sample(n_samples=10, filename='data'):
    now_date = '2018-05-20'
    DEPO = 'Депо-Ленинское'
    MEAN_TONNAGE = 2500
    vagon_json = [{} for _ in range(n_samples)]
    for i in range(n_samples):
        car_num = actual_repairs.loc[actual_repairs.index[i], 'car_num']
        # количество поломок за всю жизнь вагона
        vagon_json[i]['breaks'] = np.random.randint(low=0, high=5)
        # ??
        vagon_json[i]['distance_left'] = np.random.randint(low=0, high=200)
        # ??
        vagon_json[i]['distance_left'] = np.random.randint(low=50000, high=150000)
        # id вагона
        vagon_json[i]['id'] = car_num
        # место текущего пребывания вагона
        car_df = movements[movements['car_num'] == car_num]
        j = 0
        date = car_df['date'][car_df.index[j]]
        while date < now_date:
            j += 1
            date = car_df['date'][car_df.index[j]]
        j -= 1
        current_station_id = car_df.loc[car_df.index[j], 'station_id']
        vagon_json[i]['location'] = stations[stations['id'] == current_station_id]['name'].values[0]
        # тип вагона
        vagon_json[i]['type'] = actual_repairs[actual_repairs['car_num'] == car_num]['ct_name'].values[0]
        # ближайшие ремонты (пока что просто все ремонты)
        vagon_json[i]['maintainces'] = []
        for repair_type in REPAIR_TYPES:
            repair_date = actual_repairs[repair_type].values[0]
            maintain_dict = {}
            maintain_dict['date'] = repair_date
            maintain_dict['depo'] = DEPO  # Заменить на нормальное депо!
            maintain_dict['maintain_type'] = repair_type
            vagon_json[i]['maintainces'].append(maintain_dict)
        # ?? осталось дней до ближайшего ремонта / 2 года (спросить Эмиля)
        vagon_json[i]['part_of_time_to_next_mainaince_required'] = 0.9
        # расписание отправления/прибытия в станции
        vagon_json[i]['schedule'] = []
        car_moves = car_df[car_df['date'] > now_date]
        for k in range(7):
            move_dict = {}
            move_dict['date'] = car_moves.loc[car_moves.index[k], 'date']
            move_dict['from'] = stations[stations['id'] == car_moves.loc[car_moves.index[k], 'station_id']]['name'].values[0]
            move_dict['to'] = stations[stations['id'] == car_moves.loc[car_moves.index[k+1], 'station_id']]['name'].values[0]
            vagon_json[i]['schedule'].append(move_dict)
        # тоннаж за всё время жизни вагона
        vagon_json[i]['total_weight_transported'] = MEAN_TONNAGE
    return str(vagon_json)

In [396]:
vagons_json = make_json_sample()

In [397]:
vagons_json

"[{'location': 'Северный', 'breaks': 2, 'id': 57463309, 'total_weight_transported': 2500, 'maintainces': [{'maintain_type': 'std_kap', 'depo': 'Депо-Ленинское', 'date': '2021-06-20'}, {'maintain_type': 'std_ppr', 'depo': 'Депо-Ленинское', 'date': '2018-05-16'}, {'maintain_type': 'std_dep', 'depo': 'Депо-Ленинское', 'date': '2019-01-09'}, {'maintain_type': 'std_vogi', 'depo': 'Депо-Ленинское', 'date': '2019-04-24'}], 'schedule': [{'to': 'Милта', 'date': '2018-06-02', 'from': 'Рожок'}, {'to': 'Сосновка', 'date': '2018-06-26', 'from': 'Милта'}, {'to': 'Гатка', 'date': '2018-07-20', 'from': 'Сосновка'}, {'to': 'Милта', 'date': '2018-08-13', 'from': 'Гатка'}, {'to': 'Сосновка', 'date': '2018-09-06', 'from': 'Милта'}, {'to': 'Георгополь', 'date': '2018-09-30', 'from': 'Сосновка'}, {'to': 'Новорепное', 'date': '2018-10-24', 'from': 'Георгополь'}], 'distance_left': 110207, 'type': 'Газовые', 'part_of_time_to_next_mainaince_required': 0.9}, {'location': 'Северный', 'breaks': 0, 'id': 57456246, 